# Belegungsquote der Frauenhäuser in Deutschland

## Load source data

In [1]:
import pandas as pd
import numpy as np

In [2]:
fpath = "data/scraped/230103_datenstand.json"

In [3]:
# load data
raw = pd.read_json(fpath, orient="index")
raw["shelter_id"] = raw.index
raw = raw.reset_index(drop=True)

# clean up geometry columns
raw.loc[raw.geography.notnull(), "geometry"] = raw.loc[raw.geography.notnull(), "geography"]

In [4]:
# get keys
keys = pd.read_csv("data/helpers/free_places_key.csv")

In [5]:
# get all timeseries data
df = pd.DataFrame()

for i, row in raw.iterrows():
    
    temp = pd.DataFrame(row["data"])
    temp["shelter_id"] = row["shelter_id"]
    temp["shelter_name"] = row["title"]
    temp["latitude"] = row["geometry"]["coordinates"][1]
    temp["longitude"] = row["geometry"]["coordinates"][0]
    df = pd.concat([df, temp])

In [6]:
# reformat timeseries
df.timestamp = pd.to_datetime(df.timestamp, dayfirst=True)
df["date"] = df.timestamp.dt.date

In [7]:
# add text keys
df.loc[df.freePlaces=="", "freePlaces"] = np.nan
df.freePlaces = df.freePlaces.astype(float)
df = pd.merge(df, keys, on="freePlaces")

# fill nas
df.loc[df.description.isnull(), "description"] = "k.A."

# add simplified NAs
desc_short = {
    "Aufnahme möglich für Frauen mit 4 oder mehr Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen mit 3 Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen mit 2 Kindern":"Aufnahme möglich für Frauen mit mehreren Kindern",
    "Aufnahme möglich für Frauen ohne Kinder, Aufnahme möglich für Frauen mit 1 Kind":"Aufnahme möglich für Frauen mit bis zu einem Kind",
    "Aufnahme möglich für Frauen ohne Kinder":"Platz für Frauen ohne Kinder",
    "Keine Aufnahme möglich":"Keine Aufnahme möglich",
    "Aufnahme möglich ohne detaillierte Angabe":"Aufnahme möglich ohne detaillierte Angabe",
    "k.A.":"keine Angabe"
}

df["status"] = df.description.map(desc_short)

In [8]:
# add bundesland
geocoded = pd.read_csv("data/helpers/shelters_geocoded.csv").drop(["latitude","longitude"], axis=1)
df = pd.merge(df, geocoded, on=["shelter_name","shelter_id"])

In [9]:
# get monthyear as column
df["monthyear"] = df.timestamp.dt.to_period("M")

In [10]:
# get counts of timestamp to use as denominator
counts = df[["monthyear","timestamp"]].drop_duplicates().groupby(["monthyear"]).count().reset_index()
counts = counts.rename(columns={"timestamp":"timestamp_count"})

In [11]:
# and total counts
n = counts.timestamp_count.sum()

In [12]:
# get list of shelters with less than 80% data completeness (not in entire timestamp)
data_completeness = df.groupby(["shelter_id"]).timestamp.count().reset_index()
data_completeness["pct_data_availability"] = data_completeness["timestamp"]/n
incomplete = data_completeness.loc[data_completeness.pct_data_availability < .8, "shelter_id"].values

In [13]:
# filter out shelters with less than 80% data availability
df = df.loc[~df.shelter_id.isin(incomplete),]

## Shelter-level data by month

In [14]:
# get count of values per day
df_monthyear = df.groupby(["monthyear","shelter_name","shelter_id","bundesland","bez","gen","status","latitude","longitude"]).agg(
    status_count = ("status", "count")
).reset_index()
df_monthyear.head()

,monthyear,shelter_name,shelter_id,bundesland,bez,gen,status,latitude,longitude,status_count
0,2021-12,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,keine Angabe,53.552828,9.996640,34
1,2021-12,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Keine Aufnahme möglich,51.325188,12.373901,34
2,2021-12,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,Aufnahme möglich für Frauen mit bis zu einem Kind,50.933467,6.998638,4
3,2021-12,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,Keine Aufnahme möglich,50.933467,6.998638,30
4,2021-12,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt,Berlin,keine Angabe,52.516227,13.377745,34


In [15]:
# pivot to wide and fill values
df_monthyear_wide = pd.pivot(
    df_monthyear,
    index=["shelter_name","shelter_id","bundesland","bez","gen","monthyear"],
    columns="status",
    values="status_count"
).reset_index().replace(np.nan, 0)
df_monthyear_wide.head()

status,shelter_name,shelter_id,bundesland,bez,gen,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2021-12,0.0,0.0,0.0,0.0,0.0,34.0
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-01,0.0,0.0,0.0,0.0,0.0,92.0
2,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-02,0.0,0.0,0.0,0.0,0.0,80.0
3,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-03,0.0,0.0,0.0,0.0,0.0,91.0
4,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2022-04,0.0,0.0,0.0,0.0,0.0,85.0


In [16]:
# add total count of timestamp for percent denominator
df_monthyear_wide = pd.merge(df_monthyear_wide, counts, how="outer")
df_monthyear_wide.head()

,shelter_name,shelter_id,bundesland,bez,gen,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe,timestamp_count
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2021-12,0.0,0.0,0.0,0.0,0.0,34.0,34
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,2021-12,0.0,0.0,0.0,34.0,0.0,0.0,34
2,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,2021-12,4.0,0.0,0.0,30.0,0.0,0.0,34
3,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt,Berlin,2021-12,0.0,0.0,0.0,0.0,0.0,34.0,34
4,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,2021-12,0.0,0.0,0.0,33.0,1.0,0.0,34


In [17]:
# calculate percentages
df_monthyear_wide.iloc[:,6:-1] = df_monthyear_wide.iloc[:,6:-1].apply(lambda x : x / df_monthyear_wide.timestamp_count, axis=0)
df_monthyear_wide.head()

,shelter_name,shelter_id,bundesland,bez,gen,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe,timestamp_count
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,2021-12,0.000000,0.0,0.0,0.000000,0.000000,1.0,34
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,2021-12,0.000000,0.0,0.0,1.000000,0.000000,0.0,34
2,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,2021-12,0.117647,0.0,0.0,0.882353,0.000000,0.0,34
3,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt,Berlin,2021-12,0.000000,0.0,0.0,0.000000,0.000000,1.0,34
4,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,2021-12,0.000000,0.0,0.0,0.970588,0.029412,0.0,34


In [18]:
# drop timestamp camp and save file
df_monthyear_wide = df_monthyear_wide.drop(["timestamp_count"], axis=1)
df_monthyear_wide.to_csv("./data/cleaned/belegungsquote_nach_year_month_shelter.csv", index=False)

## Overall shelter-level summary

In [19]:
# get counts of status by shelter
df_shelter = df.groupby(["shelter_name","shelter_id","bundesland","bez","gen","status","latitude","longitude"]).agg(
    n = ("status", "count")
).reset_index()
df_shelter.head()

,shelter_name,shelter_id,bundesland,bez,gen,status,latitude,longitude,n
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,Aufnahme möglich ohne detaillierte Angabe,53.552828,9.996640,18
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,keine Angabe,53.552828,9.996640,1094
2,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Aufnahme möglich für Frauen mit bis zu einem Kind,51.325188,12.373901,151
3,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Keine Aufnahme möglich,51.325188,12.373901,270
4,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,Platz für Frauen ohne Kinder,51.325188,12.373901,691


In [20]:
# pivot to wide and fill values
df_shelter_wide = pd.pivot(
    df_shelter,
    index=["shelter_name","shelter_id","bundesland","bez","gen","latitude","longitude"],
    columns="status",
    values="n"
).reset_index().replace(np.nan, 0)

In [21]:
# calculate total unique timestamp counts
n = sum(counts.timestamp_count)

# get columns as pct
df_shelter_wide.iloc[:,-6:] = df_shelter_wide.iloc[:,-6:]/n

df_shelter_wide.head()

status,shelter_name,shelter_id,bundesland,bez,gen,latitude,longitude,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt,Hamburg,53.552828,9.996640,0.000000,0.000000,0.016187,0.000000,0.000000,0.983813
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt,Leipzig,51.325188,12.373901,0.135791,0.000000,0.000000,0.242806,0.621403,0.000000
2,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,50.933467,6.998638,0.008993,0.003597,0.000899,0.983813,0.002698,0.000000
3,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt,Berlin,52.516227,13.377745,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt,Köln,50.921396,6.995416,0.001799,0.010791,0.000000,0.982014,0.005396,0.000000


In [22]:
# save to csv
df_shelter_wide.to_csv("./data/cleaned/belegungsquote_nach_shelter.csv", index=False)

## Get all shelters with 20% or more keine Angabe

In [23]:
no_data = df_shelter_wide.loc[df_shelter_wide["keine Angabe"] >= 0.20, "shelter_id"].values

## Gesamtübersicht nach Monat

In [24]:
monthly = df_monthyear_wide.loc[~df_monthyear_wide.shelter_id.isin(no_data),] \
    .groupby(["monthyear"]) \
    .mean() \
    .reset_index()
monthly = monthly.drop(["shelter_id"], axis=1)
monthly.head()

,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,2021-12,0.016374,0.100819,0.080049,0.778957,0.016828,0.006974
1,2022-01,0.017690,0.117434,0.086637,0.735934,0.025256,0.006660
2,2022-02,0.020570,0.096359,0.069842,0.779430,0.028701,0.003398
3,2022-03,0.017284,0.072816,0.067908,0.801398,0.034621,0.004801
4,2022-04,0.021188,0.062707,0.049400,0.821416,0.033067,0.011079


In [25]:
monthly.to_csv("./data/cleaned/monthly_overview.csv", index=False)

## Gesamtübersicht nach Bundesland

In [26]:
# filter out where no data and mean of each status % by bundesland
bundesland = df_monthyear_wide.loc[~df_monthyear_wide.shelter_id.isin(no_data),] \
    .groupby(["bundesland"]) \
    .mean() \
    .reset_index()

bundesland = bundesland.drop(["shelter_id"], axis=1)
bundesland.head()

,bundesland,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,Baden-Württemberg,0.017844,0.081681,0.015875,0.848110,0.007700,0.021598
1,Bayern,0.010328,0.068311,0.068780,0.821643,0.015193,0.015433
2,Berlin,0.005322,0.009954,0.000000,0.936291,0.048434,0.000000
3,Brandenburg,0.088074,0.166747,0.029309,0.662962,0.003900,0.049008
4,Hessen,0.008258,0.024783,0.017931,0.942947,0.005879,0.000000


In [27]:
bundesland.to_csv("./data/cleaned/bundesland_overview.csv", index=False)

## Nach Bundesland und Monat

In [28]:
# filter out where no data and mean of each status % by bundesland
bundesland_monthyear = df_monthyear_wide.loc[~df_monthyear_wide.shelter_id.isin(no_data),] \
    .groupby(["bundesland", "monthyear"]) \
    .mean() \
    .reset_index()

bundesland_monthyear = bundesland_monthyear.drop(["shelter_id"], axis=1)
bundesland_monthyear.head()

,bundesland,monthyear,Aufnahme möglich für Frauen mit bis zu einem Kind,Aufnahme möglich für Frauen mit mehreren Kindern,Aufnahme möglich ohne detaillierte Angabe,Keine Aufnahme möglich,Platz für Frauen ohne Kinder,keine Angabe
0,Baden-Württemberg,2021-12,0.007353,0.180882,0.010294,0.794118,0.001471,0.005882
1,Baden-Württemberg,2022-01,0.020652,0.214674,0.016304,0.747283,0.001087,0.000000
2,Baden-Württemberg,2022-02,0.058750,0.173125,0.000000,0.748750,0.019375,0.000000
3,Baden-Württemberg,2022-03,0.017582,0.055495,0.010440,0.851648,0.008242,0.044505
4,Baden-Württemberg,2022-04,0.018235,0.094706,0.002353,0.790588,0.015294,0.067059


In [29]:
monthly_no_places = bundesland_monthyear[["bundesland","monthyear","Keine Aufnahme möglich"]]
pd.pivot(monthly_no_places, columns="bundesland", values="Keine Aufnahme möglich", index="monthyear").reset_index()

bundesland,monthyear,Baden-Württemberg,Bayern,Berlin,Brandenburg,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Nordrhein-Westfalen,Rheinland-Pfalz,Saarland,Sachsen,Sachsen-Anhalt,Schleswig-Holstein,Thüringen
0,2021-12,0.794118,0.750000,1.000000,0.632353,0.953620,0.642157,0.524064,0.861765,0.960784,0.666667,0.250000,0.400000,0.852941,0.475490
1,2022-01,0.747283,0.711957,0.876812,0.415761,0.824074,0.621377,0.699605,0.813792,0.723785,0.811594,0.274457,0.400000,0.909699,0.429348
2,2022-02,0.748750,0.811250,0.987500,0.590625,0.894907,0.631250,0.531818,0.874798,0.906250,0.666667,0.125000,0.337500,0.860577,0.635417
3,2022-03,0.851648,0.779670,0.945055,0.750000,0.889703,0.787546,0.491508,0.875222,0.914530,0.710623,0.239011,0.305495,0.892646,0.761905
4,2022-04,0.790588,0.731765,1.000000,0.505882,0.951634,0.796078,0.505882,0.935863,0.922222,0.815686,0.267647,0.642353,0.943891,0.501961
5,2022-05,0.776111,0.828889,0.848148,0.683333,0.931276,0.596296,0.619192,0.912903,0.937654,0.488889,0.375000,0.635556,0.942735,0.500000
6,2022-06,0.873529,0.883529,0.909804,0.567647,0.977778,0.750980,0.656684,0.959962,0.969281,0.760784,0.375000,0.785882,0.978281,0.500000
7,2022-07,0.821739,0.925000,0.666667,0.635870,0.974638,0.760870,0.750988,0.921985,0.949879,0.880435,0.375000,0.886957,0.954013,0.586957
8,2022-08,0.837363,0.931868,0.915751,0.750000,0.969068,0.785714,0.839161,0.945409,0.962149,0.897436,0.329670,0.953846,0.983939,0.976190
9,2022-09,0.839888,0.867416,1.000000,0.750000,0.970037,0.940075,0.820225,0.939290,0.963795,0.925094,0.643258,0.858427,0.966292,0.835206


In [30]:
bundesland_monthyear.to_csv("./data/cleaned/monthly_nach_bundesland.csv", index=False)

## Weihnachts Beispiel - 12.25.2022 um 8 Uhr

In [31]:
weihnacht = df.loc[df.timestamp==pd.to_datetime("2022-25-12 08:01:00", dayfirst=True),] # filter by day
weihnacht = weihnacht[["shelter_name","shelter_id","latitude","longitude","timestamp","description","status","gen","bez","bundesland"]]
weihnacht.to_csv("./data/cleaned/belegungsstatus_25-12-2022_8-01.csv", index=False)